In [ ]:

# Prediction of Household Electricity Consumption 1

# Step 1 (clean, multivaraite, fillna the dataset)

#Include LIbraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA

#Read Txt file
df = pd.read_csv("/content/Household Power Consumption.csv", sep=',', na_values='?', low_memory=False)
print(df.head())

print(df.isnull().sum())

# Features combine date-time in one column (datetime)
df['Datetime'] = pd.to_datetime(df['Date']+' '+df['Time'], format='mixed', dayfirst=True)
df.drop(['Date','Time'], axis=1, inplace=True) # Now we drop the date and time column
df.set_index('Datetime', inplace=True) # Now we set the datetime column as index
df.head()

#convert features into numeric
df = df.astype(float)

#Target & Features (Multivariate Setup)
features = [
    'Global_active_power',
    'Global_reactive_power',
    'Voltage',
    'Global_intensity',
    'Sub_metering_1',
    'Sub_metering_2',
    'Sub_metering_3'
]

df = df[features]

df.fillna(method='ffill', inplace=True)

#Resample
df_hourly = df.resample('H').mean()
print(df_hourly.shape)
df_hourly.head()

#Visulaisation
df_hourly['Global_active_power'].plot(figsize=(16,6))
plt.title('Hourly_Global_active_power')
plt.show()

result = adfuller(df_hourly['Global_active_power'])

print("ADF Statistic:", result[0])
print("p-value:", result[1])

df_hourly.to_csv("household_power_hourly.csv")

#Step 2 BASELINE FORECASTING MODEL (ARIMA)

series = df_hourly['Global_active_power']
train_size = int(len(series) * 0.8)
train = series[:train_size]
test = series[train_size:]

#train ARIMA model
model = ARIMA(train, order=(2,1,2))
model_fit = model.fit()

predictions = model_fit.forecast(steps=len(test))

arima_rmse = np.sqrt(mean_squared_error(test, predictions))
arima_mae = mean_absolute_error(test, predictions)

print("ARIMA RMSE:", arima_rmse)
print("ARIMA MAE:", arima_mae)

#Visuvalisation
plt.figure(figsize=(12,4))
plt.plot(test.index, test, label="Actual")
plt.plot(test.index, predictions, label="ARIMA Forecast")
plt.legend()
plt.title("ARIMA Baseline Forecast")
plt.show()


#**Step 3** LSTM

#Time-based split
data = df_hourly.values

train_size = int(len(data) * 0.7)
val_size = int(len(data) * 0.85)

train = data[:train_size]
val = data[train_size:val_size]
test = data[val_size:]

#Scaling
scaler = StandardScaler()
train_scaled = scaler.fit_transform(train)
val_scaled = scaler.transform(val)
test_scaled = scaler.transform(test)

#Sliding-window function
def create_sequences(data, lookback):
    x, y = [], []
    for i in range(len(data) - lookback):
        x.append(data[i:i+lookback, 1:])   # input features
        y.append(data[i+lookback, 0])      # target (Global_active_power)
    return np.array(x), np.array(y)

LOOKBACK = 24

x_train, y_train = create_sequences(train_scaled, LOOKBACK)
x_val, y_val = create_sequences(val_scaled, LOOKBACK)
x_test, y_test = create_sequences(test_scaled, LOOKBACK)

print(x_train.shape)
print(y_train.shape)


#**step 4 LSTM MODEL DESIGN & TRAINING**

#Minimal & Correct LSTM Architecture
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

timesteps = x_train.shape[1]
features = x_train.shape[2]

model = Sequential([
    LSTM(64, input_shape=(timesteps, features)),
    Dropout(0.2),
    Dense(1)
])

model.summary()

#Compile model
model.compile(
    optimizer='adam',
    loss='mse'
)

#Training Strategy
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

#Train the LSTM
history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=50,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

#Visualize Training
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title("LSTM Training Curve")
plt.show()

#**Step 5 HYPERPARAMETER OPTIMIZATION (LSTM)**
!pip install optuna
import optuna
import tensorflow as tf
import random

np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)

def objective(trial):

    units = trial.suggest_int("units", 32, 128)
    dropout = trial.suggest_float("dropout", 0.1, 0.5)
    lr = trial.suggest_loguniform("lr", 1e-4, 1e-2)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])

    model = Sequential([
        LSTM(units, input_shape=(x_train.shape[1], x_train.shape[2])),
        Dropout(dropout),
        Dense(1)
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
        loss='mse'
    )

    history = model.fit(
        x_train, y_train,
        validation_data=(x_val, y_val),
        epochs=30,
        batch_size=batch_size,
        verbose=0
    )

    val_loss = min(history.history['val_loss'])
    return val_loss


#Run the Optimization Study
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=25)

#Best Hyperparameters Found
print("Best Parameters:")
print(study.best_params)

print("Best Validation Loss:")
print(study.best_value)

#Train FINAL LSTM using best parameters
best_params = study.best_params

final_model = Sequential([
    LSTM(best_params['units'],
         input_shape=(x_train.shape[1], x_train.shape[2])),
    Dropout(best_params['dropout']),
    Dense(1)
])

final_model.compile(
    optimizer=tf.keras.optimizers.Adam(best_params['lr']),
    loss='mse'
)

final_model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=50,
    batch_size=best_params['batch_size'],
    callbacks=[early_stop],
    verbose=1
)

#**Step 6 FINAL MODEL EVALUATION & COMPARISON

#(ARIMA vs Optimized LSTM)

lstm_pred = final_model.predict(x_test)

lstm_rmse = np.sqrt(mean_squared_error(y_test, lstm_pred))
lstm_mae = mean_absolute_error(y_test, lstm_pred)

print("LSTM RMSE:", lstm_rmse)
print("LSTM MAE:", lstm_mae)

results = pd.DataFrame({
    "Model": ["ARIMA", "Optimized LSTM"],
    "RMSE": [arima_rmse, lstm_rmse],
    "MAE": [arima_mae, lstm_mae]
})

results

#Visual Comparison
plt.figure(figsize=(12,4))
plt.plot(y_test[:200], label="Actual")
plt.plot(lstm_pred[:200], label="LSTM Prediction")
plt.legend()
plt.title("LSTM vs Actual (Test Set)")
plt.show()



#** Step 7 MODEL INTERPRETABILITY**

#Selecting samples to explain
x_explain_reshaped = x_test[:3].reshape(x_test[:3].shape[0], -1)
background_reshaped = x_train[:50].reshape(x_train[:50].shape[0], -1)

!pip install shap
import shap


def model_predict(X):
    X= X.reshape((-1, x_train.shape[1], x_train.shape[2]))
    return final_model.predict(X)

explainer = shap.KernelExplainer(
    model_predict,
    background_reshaped
)

#Compute SHAP values
shap_values = np.squeeze(explainer.shap_values(x_explain_reshaped))
shap_agg = np.mean(np.abs(shap_values), axis=0)

shap.summary_plot(
    shap_values,
    x_explain_reshaped,
    feature_names=[f"t-{i}" for i in range(x_explain_reshaped.shape[1])]
)


#Create directories if they don't exist
import os
os.makedirs('models', exist_ok=True)
os.makedirs('results', exist_ok=True)

#Save
final_model.save("models/lstm_best.h5")
results.to_csv("results/metrics_summary.csv", index=False)
optuna_results = study.trials_dataframe()
optuna_results.to_csv("results/optuna_trials.csv", index=False)
